In [ ]:
import pandas as pd
import json

from pandas.io.json import json_normalize


quarter_1_2017 = pd.read_csv('desktop/20171.csv')
quarter_2_2017 = pd.read_csv('desktop/20172.csv')
quarter_3_2017 = pd.read_csv('desktop/20173.csv')
quarter_4_2017 = pd.read_csv('desktop/20174.csv')
quarter_1_2018 = pd.read_csv('desktop/20181.csv')

def getFrequency(a):
    '''
    input:
    a: a dataframe read by pandas.
    output:
    a series with the most frequent 10 locations within either individual pickup location or destination
    '''
    most_frequent_10_location = a['sourceid'].value_counts()
    return most_frequent_10_location[:1000]

getFrequency(quarter_1_2017)



In [ ]:
datat = quarter_1_2017[(quarter_1_2017.sourceid == 734) & (quarter_1_2017.month == 2)]

map_LA = pd.read_json('desktop/map.json', orient='records')

map_LA_header = pd.DataFrame(map_LA.features.values.tolist())
map_LA_coor = dict(pd.DataFrame(map_LA_header.geometry.tolist())['coordinates'])


locations = dict(getFrequency(quarter_1_2017))

long = []
lat = []
for i in range(len(map_LA_coor)):
    map_LA_coor[i] = map_LA_coor[i][0][0]
    print(map_LA_coor[i])

for index, data in datat.iterrows():
    value = data['mean_travel_time']/100
    
    while value > 0:
        if isinstance(map_LA_coor[data['dstid']][0], list):
            long.append(map_LA_coor[data['dstid']][0][0])
        else:
            long.append(map_LA_coor[data['dstid']][0])
        print(map_LA_coor[data['dstid']][0])
        if isinstance(map_LA_coor[data['dstid']][1], list):
            lat.append(map_LA_coor[data['dstid']][0][1])
        else:
            lat.append(map_LA_coor[data['dstid']][1])
            
        value = value - 1

print(lat)
# locations

In [ ]:
import gmplot

gmap = gmplot.GoogleMapPlotter(34.107225, -118.11683, 10)  
  
# heatmap plot heating Type 
# points on the Google map 
gmap.heatmap( lat, long ) 
print(lat)
  
gmap.draw( "desktop/map2017.html" ) 


In [ ]:
from bokeh.io import show
from bokeh.models import LogColorMapper
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

palette.reverse()


map_LA_coord = dict(pd.DataFrame(map_LA_header.geometry.tolist())['coordinates'])
for i in range(len(map_LA_coord)):
    map_LA_coord[i] = map_LA_coord[i][0]
longt = []
latt = []

for i in range(len(map_LA_coord)):
    lon = []
    la = []
    for j in range(len(map_LA_coord[i])):
        longt.append(map_LA_coord[i][j][0])
        print(map_LA_coord[i][j][1])
        la.append(map_LA_coord[i][j][1])
    longt.append(lon)
    latt.append(la)

county_xs = longt
county_ys = latt

# county_xs = [county["lons"] for county in counties.values()]
# county_ys = [county["lats"] for county in counties.values()]

#county_names = [county['name'] for county in counties.values()]
#county_rates = [unemployment[county_id] for county_id in counties]
rate_trial = [0.5 for i in range(len(map_LA_coord))]

color_mapper = LogColorMapper(palette=palette)

data=dict(
    x=county_xs,
    y=county_ys,
#     name=county_names,
    rate=rate_trial
)

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Texas Unemployment, 2009", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("(Long, Lat)", "($x, $y)")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('x', 'y', source=data,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

show(p)